# LIBRARIES

In [1]:
import Learner
from Environment import Environment
from UserCat import UserCat
from Product import Product
from Greedy_learner import *
import numpy as np
import random

# TESTING of ENVIRONMENT.py

In [3]:
users=[]
products=[]

nameofproduct= [ #name of products
    "Calabazas",
    "Hinojo",
    "Sesamo",
    "Girasol",
    "Amapola"
]

prices=[[4., 5, 6 ,7],
    [9., 10, 11, 12],
    [18., 19, 20, 21],
    [24., 25, 26, 27],
    [33., 34, 35, 36]]
#1-2 di delta, Con sovrapposizione

cost=[2,4.5,9,14,17]

#sarebbe interessante anche prendere da file il tutto così da cambiare tutto più facilmente
#calcolo i margini dai cost mi sembra più sensato e anche più veloce se dobbiamo cambiare continuamente

cost2 = np.tile(np.array([cost]).transpose(), (1, 4))
margins = np.array(prices)-cost2

Secondary_dict={           # Propongo i prodotti più simili a quello mostrato --> problemino: 2 viene mostrato quasi sempre
    "Calabazas": [1,2],
    "Hinojo": [0,2],
    "Sesamo": [1,3],
    "Girasol": [2,4],
    "Amapola": [2,3]
}

res_price_params = {
    "shape": 30,
    "scale": 3,
    "max" : 60,
    "min" : 5
}

np.random.seed(1)
probabilities = np.random.uniform(0.0, 1, (5, 5))


alphas=[1/5, 1/5, 1/5, 1/5, 1/5] 
# per ora li generiamo così, tutti uguali -> devo generare 3 diversi vettori alpha

poisson_lambda = 1
#=valore atteso del numero di prodotti acquistati (specifico per prodotto)...non dipende dal
#prodotto oltre che dallo user che dal tipo di user che

p_users = [4/9, 3/9, 2/9] #probabilità di essere un tipo di utente-> da cambiare

lambda_q = 0.5 #just my idea of lambda
#possiamo stimarlo con i dati passati provenienti dal sito -> vino tot è stato comprato 15 volte

#proviamo a pensare, ha senso vederlo come coppia? categoria-prodotto? Avrei 3 categorie *5 prodotti-> 15 lambda diversi
# ^^ Andre: secondo me ha senso avere 3 lambda_q diversi per categoria MA non per prodotto. La probabilità di continuare
#           a guardare secondo me dipende dall'utente e non dal prodotto

for i in range (5):
    products.append(Product(prices[i], i, nameofproduct[i],margins[i]))

for i in range(3):
    users.append(UserCat(alphas, res_price_params, poisson_lambda, probabilities))

Env = Environment(users, products,  lambda_q, Secondary_dict, p_users)


In [ ]:
probabilities

In [4]:
Env.optimal_reward()

(9.71937076534408, [3, 3, 3, 3, 0])

In [11]:
Env.aggregated_reward([0, 0, 0, 0, 0])

7.428526303528863

In [ ]:
Env.single_reward(0, [3, 3, 1, 0, 0])

In [ ]:
a = Env.simulate_day(10000, [3,3,1,0,0])
a
print("The margin for the price combination chosen are:", a, "\nMean reward for user", sum(a)/10000)

# Greedy Algorithm Test

In [4]:
greedy_learner = Greedy_learner(Env, 10000)
greedy_learner.run()

[56578.5, 87758.0, 38699.5, 58819.0, 64607.5]
89142.5


{'combination': [0, 0, 0, 0, 0], 'margin': 89142.5}